# A Full Training Loop in PyTorch 🔄

This notebook provides a comprehensive guide to implementing custom training loops for transformer models using PyTorch. While the Hugging Face `Trainer` API abstracts away most training details, understanding the low-level training loop gives you:

- **Full control** over the training process
- **Custom optimization** strategies
- **Advanced techniques** like gradient accumulation and mixed precision
- **Better debugging** capabilities

## Table of Contents
1. [Data Preparation](#prepare-for-training)
2. [Basic Training Loop](#the-training-loop)
3. [Modern Optimizations](#modern-training-optimizations)
4. [Evaluation Loop](#the-evaluation-loop)
5. [Accelerate Library](#supercharge-your-training-loop-with--accelerate)
6. [Practice: SST-2 Fine-tuning](#modify-the-previous-training-loop-to-fine-tune-your-model-on-the-sst-2-dataset)

In [ ]:
# Step 1: Load and Tokenize the Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Load the MRPC (Microsoft Research Paraphrase Corpus) dataset from GLUE benchmark
# MRPC: Binary classification - determine if two sentences are paraphrases
raw_datasets = load_dataset("glue", "mrpc")

# Load the BERT tokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Tokenization function for sentence pairs
def tokenize_function(example):
    """
    Tokenize pairs of sentences for paraphrase detection.
    - truncation=True: Truncate sequences longer than model's max length
    - The tokenizer automatically handles [SEP] token between sentences
    """
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Apply tokenization to all splits (train, validation, test)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# DataCollator handles dynamic padding - pads each batch to max length in that batch
# This is more efficient than padding all sequences to the same length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

## Prepare for Training

Before training, we need to prepare our dataset for PyTorch:

| Step | Action | Reason |
|------|--------|--------|
| 1 | Remove unused columns | Model only needs `input_ids`, `attention_mask`, `token_type_ids` |
| 2 | Rename `label` → `labels` | Hugging Face models expect `labels` argument |
| 3 | Set format to PyTorch | Convert from Python lists to PyTorch tensors |

In [2]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
# Step 2: Prepare Dataset for PyTorch

# Remove columns the model doesn't need (raw text and index)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# Rename 'label' to 'labels' (HuggingFace models expect 'labels')
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Convert to PyTorch tensors
tokenized_datasets.set_format("torch")

# Verify the columns are correct
print("✅ Final columns:", tokenized_datasets["train"].column_names)

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
# Verify data is now in tensor format
example = tokenized_datasets["train"][0]
print(f"Type of input_ids: {type(example['input_ids'])}")
print(f"Shape of input_ids: {example['input_ids'].shape}")

<class 'torch.Tensor'>


In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
# Step 3: Create DataLoaders
from torch.utils.data import DataLoader

# Training DataLoader with shuffling for better generalization
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,              # Shuffle training data each epoch
    batch_size=8,              # Process 8 examples at a time
    collate_fn=data_collator   # Dynamic padding per batch
)

# Evaluation DataLoader (no shuffling needed)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator
)

# Verify batch structure
for batch in train_dataloader:
    break
print("📦 Batch shapes:")
for k, v in batch.items():
    print(f"  {k}: {v.shape}")

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78])}

In [ ]:
# Step 4: Load the Pre-trained Model
from transformers import AutoModelForSequenceClassification

# Load BERT with a classification head (2 labels: paraphrase or not)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Quick sanity check - forward pass should work
outputs = model(**batch)
print(f"✅ Loss: {outputs.loss.item():.4f}")
print(f"✅ Logits shape: {outputs.logits.shape}  # (batch_size, num_labels)")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.5146, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


## Optimizer and Learning Rate Scheduler

### Choosing an Optimizer

| Optimizer | Use Case | Memory |
|-----------|----------|--------|
| `AdamW` | Standard choice for transformers | Normal |
| `AdamW8bit` | Memory-constrained training | ~50% less |
| `SGD` | Very large batch sizes | Lower |

### Learning Rate Guidelines
- **BERT-base**: `2e-5` to `5e-5`
- **Large models**: `1e-5` to `3e-5`
- **Always use warmup** for stability

```python
# Standard AdamW with weight decay (recommended)
AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# 8-bit Adam for memory efficiency (requires bitsandbytes)
import bitsandbytes as bnb
optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=5e-5)
```

In [ ]:
# Step 5: Configure Optimizer and Scheduler
from torch.optim import AdamW
from transformers import get_scheduler

# AdamW optimizer - the standard for transformer fine-tuning
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training configuration
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

# Linear learning rate scheduler with warmup
# - Warmup: Gradually increase LR from 0 to target (prevents early instability)
# - Decay: Linearly decrease LR to 0 (helps convergence)
lr_scheduler = get_scheduler(
    name="linear",                           # Linear decay after warmup
    optimizer=optimizer,
    num_warmup_steps=0,                      # No warmup (can set to ~10% of total steps)
    num_training_steps=num_training_steps
)

print(f"📊 Training Configuration:")
print(f"   Epochs: {num_epochs}")
print(f"   Batches per epoch: {len(train_dataloader)}")
print(f"   Total training steps: {num_training_steps}")

1377


## The Training Loop

The basic training loop follows these steps for each batch:

```
┌─────────────────────────────────────────────────────────────┐
│  1. Forward Pass    →  Compute predictions and loss        │
│  2. Backward Pass   →  Calculate gradients                 │
│  3. Optimizer Step  →  Update model weights                │
│  4. Scheduler Step  →  Adjust learning rate                │
│  5. Zero Gradients  →  Clear gradients for next iteration  │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Step 6: Setup Device (GPU/CPU)
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(f"🖥️ Training on: {device}")
if device.type == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

device(type='cuda')

In [ ]:
# Step 7: Basic Training Loop
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps), desc="Training")

model.train()  # Set model to training mode (enables dropout, etc.)

for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Move batch to device (GPU/CPU)
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass - compute predictions and loss
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward pass - compute gradients
        loss.backward()
        
        # Optimizer step - update weights using gradients
        optimizer.step()
        
        # Scheduler step - adjust learning rate
        lr_scheduler.step()
        
        # Zero gradients - clear for next iteration
        optimizer.zero_grad()
        
        # Update progress bar
        progress_bar.update(1)
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

  0%|          | 0/1377 [00:00<?, ?it/s]

In [ ]:
# Step 8: Evaluation Loop
import evaluate

# Load the GLUE MRPC metric (F1 and Accuracy)
metric = evaluate.load("glue", "mrpc")

model.eval()  # Set model to evaluation mode (disables dropout)

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    
    # No gradient computation needed for evaluation
    with torch.no_grad():
        outputs = model(**batch)
    
    # Get predictions from logits
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    
    # Add batch results to metric
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Compute final metrics
results = metric.compute()
print(f"📈 Evaluation Results:")
print(f"   Accuracy: {results['accuracy']:.4f}")
print(f"   F1 Score: {results['f1']:.4f}")

{'accuracy': 0.8602941176470589, 'f1': 0.9025641025641026}

---

## Modern Training Optimizations 🚀

To make training more efficient and stable, we can add these techniques:

| Technique | Benefit | When to Use |
|-----------|---------|-------------|
| **Gradient Clipping** | Prevents exploding gradients | Always recommended |
| **Mixed Precision (FP16)** | 2x faster, 50% less memory | GPU with Tensor Cores |
| **Gradient Accumulation** | Simulate larger batch sizes | Limited GPU memory |
| **Checkpointing** | Resume interrupted training | Long training runs |

### How They Work Together:

```
┌────────────────────────────────────────────────────────────────────┐
│  for step, batch in enumerate(dataloader):                        │
│      with autocast('cuda'):           # ← Mixed Precision         │
│          loss = model(**batch).loss                                │
│          loss = loss / accum_steps    # ← Gradient Accumulation   │
│                                                                    │
│      scaler.scale(loss).backward()    # ← Scaled Backward         │
│                                                                    │
│      if (step + 1) % accum_steps == 0:                            │
│          scaler.unscale_(optimizer)                                │
│          clip_grad_norm_(params, 1.0) # ← Gradient Clipping       │
│          scaler.step(optimizer)                                    │
│          scaler.update()                                           │
│          optimizer.zero_grad()                                     │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Advanced Training Loop with All Optimizations
from tqdm.auto import tqdm
import torch
from torch.amp import GradScaler, autocast

# Configuration
gradient_accumulation_steps = 4              # Accumulate gradients over 4 batches
effective_batch_size = 8 * gradient_accumulation_steps  # = 32

# Mixed Precision: GradScaler handles loss scaling to prevent underflow in FP16
scaler = GradScaler('cuda')

# Adjust progress bar for accumulated steps
total_optimization_steps = num_training_steps // gradient_accumulation_steps
progress_bar = tqdm(range(total_optimization_steps), desc="Training (Optimized)")

model.train()

for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Mixed Precision: autocast automatically uses FP16 where safe
        with autocast('cuda'):
            outputs = model(**batch)
            # Scale loss by accumulation steps for correct gradient magnitude
            loss = outputs.loss / gradient_accumulation_steps
        
        # Backward pass with gradient scaling (prevents FP16 underflow)
        scaler.scale(loss).backward()
        
        # Only update weights after accumulating enough gradients
        if (step + 1) % gradient_accumulation_steps == 0:
            # Unscale gradients before clipping
            scaler.unscale_(optimizer)
            
            # Gradient Clipping: Prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Optimizer step with scaled gradients
            scaler.step(optimizer)
            scaler.update()
            
            # Learning rate scheduler step
            lr_scheduler.step()
            
            # Clear gradients for next accumulation cycle
            optimizer.zero_grad()
            
            # Update progress
            progress_bar.update(1)
            progress_bar.set_postfix({
                "loss": f"{loss.item() * gradient_accumulation_steps:.4f}",
                "lr": f"{lr_scheduler.get_last_lr()[0]:.2e}"
            })

print(f"\n✅ Training complete! Effective batch size: {effective_batch_size}")

  0%|          | 0/344 [00:00<?, ?it/s]

---

## The Evaluation Loop

After training, we evaluate the model on the validation set:

```python
model.eval()                    # Disable dropout, batch norm in eval mode
with torch.no_grad():           # No gradient computation needed
    outputs = model(**batch)
predictions = torch.argmax(outputs.logits, dim=-1)
```

In [ ]:
# Install the evaluate library if not already installed
!pip install evaluate -q

In [ ]:
# Evaluation after optimized training
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

results = metric.compute()
print(f"📈 Final Evaluation Results:")
print(f"   Accuracy: {results['accuracy']:.4f}")
print(f"   F1 Score: {results['f1']:.4f}")

{'accuracy': 0.8602941176470589, 'f1': 0.9025641025641026}

---

## Supercharge Your Training Loop with 🤗 Accelerate

The **Accelerate** library provides a simple way to run the same training code on:
- Single GPU
- Multiple GPUs (Data Parallel)
- TPUs
- Mixed precision

### Key Benefits:
| Feature | Benefit |
|---------|---------|
| `accelerator.prepare()` | Automatically handles device placement |
| `accelerator.backward()` | Handles distributed gradients |
| `accelerator.gather_for_metrics()` | Collects predictions across devices |

### Minimal Code Changes:
```python
from accelerate import Accelerator
accelerator = Accelerator()

# Wrap your objects
model, optimizer, train_dl, eval_dl = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# Replace loss.backward() with:
accelerator.backward(loss)
```

In [ ]:
# Complete Training Script with Accelerate
from accelerate import Accelerator
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, get_scheduler
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader
import torch
import evaluate
from tqdm.auto import tqdm

# ============ Data Preparation ============
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.with_format("torch")

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

# ============ Accelerator Setup ============
accelerator = Accelerator()  # Automatically detects available hardware

# ============ Model & Optimizer ============
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# Prepare for distributed training (handles device placement automatically)
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

# ============ Scheduler ============
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# ============ Training Loop ============
progress_bar = tqdm(range(num_training_steps), desc="Training with Accelerate")

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        
        # Use accelerator.backward() instead of loss.backward()
        accelerator.backward(loss)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# ============ Evaluation ============
metric = evaluate.load("glue", "mrpc")
model.eval()

for batch in eval_dl:
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    
    # Gather predictions from all devices (important for distributed training)
    predictions, references = accelerator.gather_for_metrics(
        (predictions, batch["labels"])
    )
    metric.add_batch(predictions=predictions, references=references)

eval_metric = metric.compute()
print(f"\n📈 Evaluation Results: Accuracy={eval_metric['accuracy']:.4f}, F1={eval_metric['f1']:.4f}")

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

Evaluation Results: {'accuracy': 0.8651960784313726, 'f1': 0.9066213921901528}


In [ ]:
# Install required packages
!pip install datasets evaluate accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.12.0
    Uninstalling fsspec-2025.12.0:
   

In [ ]:
%%writefile train_script.py
"""
Distributed Training Script with Accelerate

Run with: accelerate launch train_script.py
Configure with: accelerate config
"""
from accelerate import Accelerator
from torch.optim import AdamW
from transformers import (
    AutoModelForSequenceClassification,
    get_scheduler,
    AutoTokenizer,
    default_data_collator
)
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
import evaluate
from tqdm.auto import tqdm

def main():
    # ============ Data Preparation ============
    raw_datasets = load_dataset("glue", "mrpc")
    checkpoint = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    def tokenize_function(example):
        return tokenizer(
            example["sentence1"],
            example["sentence2"],
            truncation=True,
            padding="max_length",
            max_length=128
        )
    
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets = tokenized_datasets.with_format("torch")
    
    train_dataloader = DataLoader(
        tokenized_datasets["train"],
        shuffle=True,
        batch_size=8,
        collate_fn=default_data_collator
    )
    eval_dataloader = DataLoader(
        tokenized_datasets["validation"],
        batch_size=8,
        collate_fn=default_data_collator
    )
    
    # ============ Accelerator Setup ============
    accelerator = Accelerator()
    
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    optimizer = AdamW(model.parameters(), lr=3e-5)
    
    train_dl, eval_dl, model, optimizer = accelerator.prepare(
        train_dataloader, eval_dataloader, model, optimizer
    )
    
    # ============ Training Configuration ============
    num_epochs = 3
    num_training_steps = num_epochs * len(train_dl)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    
    # Only show progress on main process
    if accelerator.is_main_process:
        progress_bar = tqdm(range(num_training_steps), desc="Training")
    
    # ============ Training Loop ============
    model.train()
    for epoch in range(num_epochs):
        for batch in train_dl:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            
            if accelerator.is_main_process:
                progress_bar.update(1)
    
    # ============ Evaluation ============
    if accelerator.is_main_process:
        print("\n🔍 Starting evaluation...")
    
    metric = evaluate.load("glue", "mrpc")
    model.eval()
    
    for batch in eval_dl:
        with torch.no_grad():
            outputs = model(**batch)
        
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        predictions, references = accelerator.gather_for_metrics(
            (predictions, batch["labels"])
        )
        metric.add_batch(predictions=predictions, references=references)
    
    if accelerator.is_main_process:
        eval_metric = metric.compute()
        print(f"📈 Results: Accuracy={eval_metric['accuracy']:.4f}, F1={eval_metric['f1']:.4f}")

if __name__ == "__main__":
    main()

Overwriting train_script.py


### Running Distributed Training

```bash
# Configure accelerate (choose hardware: CPU, GPU, multi-GPU, TPU)
accelerate config

# Launch training
accelerate launch train_script.py
```

In [5]:
!accelerate launch train_script.py

	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
README.md: 35.3kB [00:00, 93.5MB/s]
mrpc/train-00000-of-00001.parquet: 100% 649k/649k [00:01<00:00, 404kB/s]  
mrpc/validation-00000-of-00001.parquet: 100% 75.7k/75.7k [00:00<00:00, 104kB/s]
mrpc/test-00000-of-00001.parquet: 100% 308k/308k [00:00<00:00, 760kB/s]
Generating train split: 100% 3668/3668 [00:00<00:00, 6543

---

## 📝 Exercise: Fine-tune on SST-2 Dataset

Now it's your turn! Modify the training loop to fine-tune BERT on the **SST-2** (Stanford Sentiment Treebank) dataset:

- **Task**: Binary sentiment classification (positive/negative)
- **Dataset**: Single sentences with sentiment labels
- **Key difference**: Only one sentence per example (not sentence pairs like MRPC)

In [ ]:
# Solution: Complete SST-2 Fine-tuning Pipeline
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import evaluate

# ============ Setup ============
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"🖥️ Using device: {device}")

# ============ Data Preparation ============
# SST-2: Stanford Sentiment Treebank (binary sentiment classification)
raw_datasets = load_dataset("glue", "sst2")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Note: SST-2 has single sentences (unlike MRPC which has sentence pairs)
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Prepare dataset
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

print(f"📊 Dataset: SST-2")
print(f"   Training samples: {len(tokenized_datasets['train'])}")
print(f"   Validation samples: {len(tokenized_datasets['validation'])}")

# ============ Model Setup ============
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.to(device)

# ============ Training Configuration ============
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 1  # SST-2 is larger, so 1 epoch is often enough for demo
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# ============ Training Loop ============
progress_bar = tqdm(range(num_training_steps), desc="Fine-tuning on SST-2")

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

print("\n✅ Training complete! Starting evaluation...")

# ============ Evaluation ============
metric = evaluate.load("glue", "sst2")
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

final_score = metric.compute()
print(f"\n📈 SST-2 Evaluation Results:")
print(f"   Accuracy: {final_score['accuracy']:.4f}")

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/8419 [00:00<?, ?it/s]

Training Finished! Starting Evaluation...
{'accuracy': 0.9162844036697247}
